In [29]:
desired_system_capacity = 750
option_table_path = 'example-diesel-generator-by-capacity.csv'
capacity_column = 'capacity_in_kw'

In [30]:
from pandas import read_csv
option_table = read_csv(option_table_path)
option_table

,Capacity in kW,Raw Cost,Installation Cost as Percent of Raw Cost,Maintenance Cost per Year as Percent of Raw Cost,Lifetime in Years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [31]:
from invisibleroads_macros.table import normalize_column_name
option_table.columns = [normalize_column_name(x, '_') for x in option_table.columns]
option_table.columns

Index([u'capacity_in_kw', u'raw_cost',
       u'installation_cost_as_percent_of_raw_cost',
       u'maintenance_cost_per_year_as_percent_of_raw_cost',
       u'lifetime_in_years'],
      dtype='object')

In [32]:
desired_system_capacity

750

In [33]:
x = (option_table['capacity_in_kw'] - desired_system_capacity).abs()
x

0    250
1    250
Name: capacity_in_kw, dtype: int64

In [34]:
option_table.ix[x.argsort()]

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [40]:
# We should pre sort the tables to normalize them
option_table.sort_values([
    'capacity_in_kw',
], ascending=True)

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [52]:
option_table.sort_values('capacity_in_kw', ascending=False`)

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
1,1000,100000,25,1,5
0,500,52500,25,1,5


In [ ]:
# We should assert non-duplicates of capacity_in_kw

In [43]:
len(set(option_table['capacity_in_kw']))

2

In [49]:
len(option_table)

2

In [50]:
from infrastructure_planning.exceptions import InvalidData

In [ ]:
if len(option_table) != len(set(option_table['capacity_in_kw'])):
    raise InvalidData

In [48]:
assert 1 == 2, 'hey'

AssertionError: hey

In [ ]:
'capacity_in_kw',
'raw_cost',
'installation_cost_as_percent_of_raw_cost',
'maintenance_cost_per_year_as_percent_of_raw_cost',
'lifetime_in_years',

In [ ]:
def prepare_system_cost(desired_system_capacity, option_table, capacity_column):
    
    
    
    return {
        'actual_system_' + capacity_column: desired_system_capacity,
        'raw_cost': raw_cost,
        'installation_cost': installation_cost,
        'maintenance_cost_per_year': maintenance_cost_per_year,
        'replacement_cost_per_year': replacement_cost_per_year,
    }

In [ ]:
"""
    Name initial variables
    Draft table
    Specify final variables

    Fix oversizing system capacity
        Draft system capacity algorithm
            _ Get generator capacity
            _ Get generator raw cost
            _ Get generator installation cost as percent of raw cost
            _ Get maintenance cost per year as percent of raw cost
            _ Get desired system capacity
            _ Add system capacity safety factor
            Compute raw cost per kw
            Pick nearest generator capacity and use those costs per kw
            Round actual capacity to integer
"""

In [ ]:
def prepare_system_cost(
        desired_system_capacity, option_table, capacity_column):
    t = option_table
    # Select option
    eligible_t = t[t[capacity_column] < desired_system_capacity]
    if len(eligible_t):
        t = eligible_t
        # Choose the largest capacity from eligible types
        selected_t = t[t[capacity_column] == t[capacity_column].max()]
    else:
        # Choose the smallest capacity from all types
        selected_t = t[t[capacity_column] == t[capacity_column].min()]
    selected = selected_t.ix[selected_t.index[0]]
    # Get capacity and count
    selected_capacity = selected[capacity_column]
    selected_count = int(ceil(divide_safely(
        desired_system_capacity, selected_capacity,
        ExpectedPositive('selected_capacity'))))
    actual_system_capacity = selected_capacity * selected_count
    # Get costs
    installation_lm_cost = selected_count * selected['installation_lm_cost']
    maintenance_lm_cost_per_year = selected_count * selected[
        'maintenance_lm_cost_per_year']
    replacement_lm_cost_per_year = divide_safely(
        installation_lm_cost, selected['lifetime_in_years'],
        ExpectedPositive('lifetime_in_years'))
    return {
        'desired_system_' + capacity_column: desired_system_capacity,
        'selected_' + capacity_column: selected_capacity,
        'selected_count': selected_count,
        'actual_system_' + capacity_column: actual_system_capacity,
        'installation_lm_cost': installation_lm_cost,
        'maintenance_lm_cost_per_year': maintenance_lm_cost_per_year,
        'replacement_lm_cost_per_year': replacement_lm_cost_per_year,
    }